# Train big dataset on 2 gpus

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Create the necesary json files

In [4]:
%%writefile ./projects/TridentNet_5class/xml2coco.py
from lxml import etree
import json
import os
import matplotlib.image as mpimg

# def xml2dicts(anno_dir,file,anno_id,categories_list):
# def xml2dicts(anno_dir,img_dir,file,anno_id,categories_list,y):
def xml2dicts(anno_dir,img_dir,file,anno_id,categories_list):
    '''
    Used by function `split_dataset', 
    Convert the xml annotation file into the format function `split_dataset' accept
    Args:
        anno_dir(string): the path of the dataset annitations, e.g. 'simdata/val/Annotations'
        img_dir(string): the path of the dataset images, 
            to read the image shape
            (hardcoded as (380,190) in xml files but that's not ture)
        file: the filename of the xml annotation file
        anno_id: the annotation id of the first annotation
        categories_list: the list of the categories
    Returns:
        tuple: the tuple contains 1 dictionary, 1 list and 1 int:
            dictionary: the image dictionary in COCO format
            list: the list of the annotation dictionaries in COCO format
            e.g. ({image_dict}, [{anno_dict1}, {anno_dict2}, ...])
    '''
    img_record = {}
    
    img_name = file[:-3]+'jpg'
    
    image_id = ''
    for i in filter(str.isdigit, img_name):
        image_id += i
    img_record['file_name'] = img_name
    img_record['id']=int(image_id)
    
#     if img_name in y:
#         img_record['width'] = 380
#         img_record['height'] = 760
#     else:
    img_record['width'] = 190
    img_record['height'] = 380
#     img = mpimg.imread(os.path.join(img_dir, img_name))
#     (height, width) =  img.shape
#     img_record['width'] = width
#     img_record['height'] = height
             
    tree=etree.parse(os.path.join(anno_dir,file))
    root=tree.getroot()
    bboxes=[]
    for c in root.iter():
        if c.tag=='object':
            bboxes.append(c)
    
    annotations = []           
    for bbox in bboxes:
        anno_record ={}
        anno_record['id'] = anno_id
        anno_id += 1
        anno_record['image_id'] = img_record['id']
        anno_record['iscrowd'] = 0
        box=[]
        for b in bbox.iter(): 
            if b.tag == 'name':
                anno_record['category_id'] = categories_list.index(b.text)
            elif b.tag == 'xmin':
                xmin = int(b.text)-1
            elif b.tag == 'ymin':
                ymin = int(b.text)-1
            elif b.tag == 'xmax':
                xmax = int(b.text)-1
            elif b.tag == 'ymax':
                ymax = int(b.text)-1
        if file[:5]=='Train':
            xmin = int(xmin/2)
            ymin = int(ymin/2)
            xmax = int(xmax/2)
            ymax = int(ymax/2)
        width = xmax - xmin +1
        height = ymax - ymin + 1
        anno_record['bbox'] = [xmin, ymin, width, height]
        anno_record['area'] = float(width*height)
        annotations.append(anno_record)
    return img_record, annotations
    
def split_dataset(anno_dir, img_dir, output_dir = None):
    '''
    Convert the xml dataset annotations into 2 COCO format ground truth [train, test] annotations json files,
    so as to call `COCOEvaluator` and function `register_coco_instances`
    Args:
        anno_dir(string): the path of the dataset annitations, e.g. 'simdata/val/Annotations'
        img_dir(string): the path of the dataset images, to make sure that there is a image corresponding to the xml file
        output_dir(string): the directory to save the 2 json files: train_anno.json & test_anno.json
    '''
#     with open('./projects/TridentNet_5class/dataset/big_image_760x380.txt', 'r') as f:
#         x = f.readlines()
#     y = []
#     for i in x:
#         y.append(i[:-27])
        
    xml_files=os.listdir(anno_dir)
    img_files=os.listdir(img_dir)
    
    json_train = {}
    json_test = {}
    
    categories = []
    categories_list = ['gun', 'knife', 'cellphone', 'explosive', 'object']
    for i,j in enumerate(categories_list):
        category_dict = {}
        category_dict['id'] = i
        category_dict['name'] = j
        categories.append(category_dict)
    json_train['categories'] = categories
    json_test['categories'] = categories
    
    train_images = []
    test_images = []

    train_annotations = []
    test_annotations = []
    
    anno_id = 0
    for file in xml_files:
        # becasue I'm using Jupyterlab
        # which always creates the boring cache file/directory
        if not file[-3:] == 'xml':
            continue
        elif file[:-3]+'jpg' not in img_files:
            continue
        elif file[:8] == 'Train_12':
            anno_single_file = xml2dicts(anno_dir,img_dir,file,anno_id,categories_list)
            test_images.append(anno_single_file[0])
            test_annotations.extend(anno_single_file[1])
            anno_id += len(anno_single_file[1])
        else:
            anno_single_file = xml2dicts(anno_dir,img_dir,file,anno_id,categories_list)
            train_images.append(anno_single_file[0])
            train_annotations.extend(anno_single_file[1])
            anno_id += len(anno_single_file[1])
            
    json_train['images'] = train_images
    json_test['images'] = test_images
    json_train['annotations'] = train_annotations
    json_test['annotations'] = test_annotations
    
    json_train_file = json.dumps(json_train)
    json_test_file = json.dumps(json_test)
    if output_dir == None:
        output_dir = anno_dir
    with open(os.path.join(output_dir,'train_annotations.json'),'w') as f:
        f.write(json_train_file)
    with open(os.path.join(output_dir,'test_annotations.json'),'w') as f:
        f.write(json_test_file)

Overwriting ./projects/TridentNet_5class/xml2coco.py


In [5]:
from luckyDiary.projects.TridentNet_5class.xml2coco import split_dataset
split_dataset('../sim_data_center/multi_class_detection/down/Annotations','../sim_data_center/multi_class_detection/down/JPEGImages','./projects/TridentNet_5class/dataset/')

## Train

In [13]:
%%writefile ./projects/TridentNet_5class/train_net.py
import os
from detectron2.data import MetadataCatalog
from detectron2.config import CfgNode as CN
from detectron2.data.datasets import register_coco_instances
from detectron2.config import get_cfg
from projects.TridentNet.tridentnet import add_tridentnet_config
from detectron2.engine import default_setup, DefaultTrainer, default_argument_parser, launch
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import COCOEvaluator, verify_results
import detectron2.utils.comm as comm

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    add_tridentnet_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.MODEL.PIXEL_MEAN = [28, 28, 28]
    cfg.MODEL.RPN.IOU_THRESHOLDS = [0.05, 0.3]
    cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.3]
    cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[8,16,32,64,128]]
    cfg.SOLVER.MAX_ITER = 1200000
    cfg.DATASETS.TRAIN = ('sim_multi_train',)
    cfg.DATASETS.TEST = ('sim_multi_test',)
    cfg.INPUT.CROP = CN({"ENABLED": True})
    cfg.INPUT.CROP.TYPE = "relative_range"
    cfg.INPUT.CROP.SIZE = [0.75, 0.95]
    cfg.TEST.EVAL_PERIOD = 50000
    cfg.MODEL.WEIGHTS = './output/model_1054999.pth'
    cfg.SOLVER.BASE_LR = 0.0025
    cfg.freeze()
    default_setup(cfg, args)
    return cfg


def main(args):
    # register the datasets
#     register_coco_instances('sim_multi_train', {},\
#                         '../../small_multi/train_annotations.json', \
#                         '../../../sim_data_center/multi_class_detection/down/JPEGImages/')
#     register_coco_instances('sim_multi_test', {},\
#                         '../../small_multi/test_annotations.json', \
#                         '../../../sim_data_center/multi_class_detection/down/JPEGImages/')
    register_coco_instances('sim_multi_train', {},\
                        './dataset/train_annotations.json', \
                        '../../../sim_data_center/multi_class_detection/down/JPEGImages/')
    register_coco_instances('sim_multi_test', {},\
                        './dataset/test_annotations.json', \
                        '../../../sim_data_center/multi_class_detection/down/JPEGImages/')
#     register_coco_instances('sim_multi_train', {},\
#                         './projects/TridentNet_5class/dataset/train_annotations.json', \
#                         '../sim_data_center/multi_class_detection/down/JPEGImages/')
#     register_coco_instances('sim_multi_test', {},\
#                         './projects/TridentNet_5class/dataset/test_annotations.json', \
#                         '../sim_data_center/multi_class_detection/down/JPEGImages/')
    cfg = setup(args)

    if args.eval_only:
        model = Trainer.build_model(cfg)
        cfg.defrost()
        cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#         cfg.MODEL.RPN.IOU_THRESHOLDS = [0.05, 0.3]
#         cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.2]
        cfg.freeze()
        DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=args.resume
        )
        res = Trainer.test(cfg, model)
        if comm.is_main_process():
            verify_results(cfg, res)
        return res

    trainer = Trainer(cfg)
    trainer.resume_or_load(resume=args.resume)
    return trainer.train()

if __name__ == "__main__":
    # specify the gpus to use
    os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'
    args = default_argument_parser().parse_args()
    print("Command Line Args:", args)
    launch(
        main,
        args.num_gpus,
        num_machines=args.num_machines,
        machine_rank=args.machine_rank,
        dist_url=args.dist_url,
        args=(args,),
    )

Overwriting ./projects/TridentNet_5class/train_net.py


In [43]:
int(21.5)

21

In [44]:
int(43/2)

21

In [4]:
round(4.5)

4

In [5]:
round(4.8)

5

In [6]:
round(4.51)

5

In [7]:
round(4.4)

4